# Drop city and smart location

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('/Users/kavisha/Documents/Springboard/listingsNY.csv', low_memory=False)
pd.options.display.max_columns = 100

In [3]:
data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2515,https://www.airbnb.com/rooms/2515,20181206022948,2018-12-06,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...,none,NaN,Please no cooking at night but you can warm up...,Subway 2.3.B.C. at 110th street around the cor...,Guests will have their PRIVATE BATHROOM (NOTE:...,We will have a list of Harlem restaurants and ...,no-smoking/please take off your shoes: cleanin...,NaN,NaN,https://a0.muscache.com/im/pictures/d0489e42-4...,NaN,2758,https://www.airbnb.com/users/show/2758,Steph,2008-09-06,"New York, New York, United States",loves to host and welcome travelers from arou...,within a few hours,70%,NaN,t,https://a0.muscache.com/im/users/2758/profile_...,https://a0.muscache.com/im/users/2758/profile_...,Harlem,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"New York, NY, United States",Harlem,Harlem,Manhattan,New York,NY,10026,New York,"New York, NY",US,United States,40.799205,-73.953676,t,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$59.00,$720.00,"$1,690.00",$0.00,$60.00,2,$39.00,2,21,today,t,3,12,23,298,2018-12-06,175,2008-10-13,2018-11-18,93.0,9.0,9.0,10.0,9.0,9.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,3,1.42
1,21456,https://www.airbnb.com/rooms/21456,20181206022948,2018-12-06,Light-filled classic Central Park,"An adorable, classic, clean, light-filled one-...","An adorable, classic, clean, light-filled one-...","An adorable, classic, clean, light-filled one-...",none,"Diverse. Great coffee shops and restaurants, n...",NaN,"Yes! B, C, and 1 trains are within 5 minute w...",Guests have access to the whole apartment: wi...,I interact with guests upon arrival & departur...,"Sorry, no pets and no smoking, please. No pa...",NaN,NaN,https://a0.muscache.com/im/pictures/111808/a94...,NaN,42032,https://www.airbnb.com/users/show/42032,Dana,2009-09-29,"New York, New York, United States","I'm an arts consultant, personal trainer and a...",within a day,80%,NaN,f,https://a0.muscache.com/im/users/42032/profile...,https://a0.muscache.com/im/users/42032/profile...,Upper West Side,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,"New York, NY, United States",Upper West Side,Upper West Side,Manhattan,New York,NY,10025,New York,"New York, NY",US,United States,40.797642,-73.961775,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$140.00,$900.00,NaN,

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49056 entries, 0 to 49055
Data columns (total 96 columns):
id                                  49056 non-null int64
listing_url                         49056 non-null object
scrape_id                           49056 non-null int64
last_scraped                        49056 non-null object
name                                49039 non-null object
summary                             47140 non-null object
space                               34354 non-null object
description                         48522 non-null object
experiences_offered                 49056 non-null object
neighborhood_overview               29889 non-null object
notes                               20013 non-null object
transit                             30935 non-null object
access                              28549 non-null object
interaction                         27835 non-null object
house_rules                         29660 non-null object
thumbnail_url          

There are a total of 96 columns in the dataset.The fist step of cleaning up the data is to remove unnecessary columns from the dataframe. We will make a set of columns which will be dropped to get a cleaner dataframe with only data which will be useful featured for modelling.

The columns where >50% of the values are null can be deleted, because we don't have enough information to fill in the missing values. Total rows (listings) are 49056, and half of that is 24528. Accordingly, all columns with more than 24528 null values will be removed.

In [6]:
# Selecting only the columns with less than 50% non-null values and assigning them to the list of columns to be deleted
# from the dataframe.
cols_del=set(data.loc[data.count(1) < data.shape[1]/2, data.count(0) < data.shape[0]/2].columns)

Next, we can check which columns have redundant data, i.e. which columns have just one value being repeated in each row. These columns are redundant and won't be useful features for modelling.

In [7]:
count_of_cols=data.apply(lambda x: True if x.nunique()==1 else False)
redundant_data_cols=set(count_of_cols[count_of_cols==True].index)
redundant_data_cols
cols_del.update(redundant_data_cols)

There are several columns with urls of listing, host profile, pictures, etc. These url columns will not be useful features in regression modelling. So, we can drop these columns.

In [8]:
url_cols = set([col for col in data.columns if 'url' in col])
cols_del.update(url_cols)

Next, we need to check if there are any listings which are being repeated. We can do this by checking for duplicate values in ID and name columns

In [9]:
print(data['id'].value_counts().head())
print(data['name'].value_counts().head())

3344383     1
2383233     1
27127045    1
6765965     1
30353698    1
Name: id, dtype: int64
New york Multi-unit building    31
Hillside Hotel                  18
New York Apartment              17
Home away from home             13
Brooklyn Apartment              13
Name: name, dtype: int64


This shows us that each of the listing has an unique ID associated with it. There are listings with the same name but different IDs. These can still be considered as seperate listings. One reason for several listings with same name could be that they are: <br>
> accomodation in a hotel/apartment building - i.e. same place but several listings <br>
generic names (such are private room/ suite)>

But, we do not have enough evidence to remove any of the rows due to redundancy at this point

We can remove the ID columns since it won't be a useful feature in modelling as they are unique for each listing.

In [10]:
# Adding id to the set cols_del
cols_del.add('id')

Exploring summary, space and description

In [11]:
data[['summary','space','description']].head()

,summary,space,description
0,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...
1,"An adorable, classic, clean, light-filled one-...","An adorable, classic, clean, light-filled one-...","An adorable, classic, clean, light-filled one-..."
2,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...
3,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ..."
4,A great space in a beautiful neighborhood- min...,Nice room in a spacious pre-war apartment in u...,A great space in a beautiful neighborhood- min...


Based on inspecting the columns, we find out that description is merely text from summary and space columns put together. Description has the least number of null values. So, for further analysis, we will retain the description column and drop summary and space columns. We will also remove the rows where decription has null value since that would be hard to fill in. We will remove those rows once we are done analyzing the data.

In [12]:
cols_del.update(['space','summary'])

Next, we will exploring the following five columns and check how many null values each contains. These all columns could be turned into True/False columns and used as features for modelling i.e. if the value is not null, then its True, and if it's null then its false. 
NLP can also be performed on these columns but for initial modelling the true/false approch will be implemented.

In [13]:
print(data['neighborhood_overview'].notnull().value_counts())
print(data['transit'].notnull().value_counts())
print(data['access'].notnull().value_counts())
print(data['interaction'].notnull().value_counts())
print(data['house_rules'].notnull().value_counts())

True     29889
False    19167
Name: neighborhood_overview, dtype: int64
True     30935
False    18121
Name: transit, dtype: int64
True     28549
False    20507
Name: access, dtype: int64
True     27835
False    21221
Name: interaction, dtype: int64
True     29660
False    19396
Name: house_rules, dtype: int64


Assessing the host id and host name columns 

In [14]:
print(data['host_id'].value_counts().head())
print(data['host_name'].value_counts().head())

107434423    174
30283594     130
12243051      98
22541573      98
16098958      92
Name: host_id, dtype: int64
Michael    421
David      409
John       299
Alex       274
Sarah      248
Name: host_name, dtype: int64


From exploring the host_id and host_name columns we see that there are certain repeating host_ids as well as host names. <br>
> Repeating host_id reperesents one host having multiple listings. <br>
Repeating host_names could represent one host having multiple listings. But, multiple different host could have the same name if it's a common name. e.g: 421 listings have the host_name listed as 'Michael' and there could be several host's named Michael. <br>
    
We can drop host_name as that won't be a useful feature for the regression modelling. But, host_id can be an interesting feture to include in the modelling.

In [15]:
#Adding host_name column to the cols_del set
cols_del.add('host_name')

Inspectin the 3 columns: neighbourhood, host_neighbourhood and neighbourhood_cleansed.

In [16]:
data[['neighbourhood_cleansed','host_neighbourhood','neighbourhood']].head(10)

,neighbourhood_cleansed,host_neighbourhood,neighbourhood
0,Harlem,Harlem,Harlem
1,Upper West Side,Upper West Side,Upper West Side
2,Kensington,Gravesend,Kensington
3,Midtown,Midtown,Midtown
4,Harlem,Hamilton Heights,Hamilton Heights
5,Williamsburg,East Williamsburg,Williamsburg
6,Chelsea,Meatpacking District,Meatpacking District
7,Harlem,Harlem,Harlem
8,Bedford-Stuyvesant,Bedford-Stuyvesant,NaN
9,Harlem,Hamilton Heights,NaN


neighbourhood_cleansed has no null values. It can be assumed that column has been created by the dataset provider (insideairbnb) by using the latter columns. For the sake of modelling, we can remove the latter two columns and only proceed with the neighbourhood_cleansed column.

In [17]:
# Adding host_neighbourhood and neighbourhood cleansed to cols_del
cols_del.update(['host_neighbourhood'],['neighbourhood'])

City and smart_location are decriptive columns with many unique values. Since we have other location features in the dataset, we can decide to drop the 2 colums.

In [18]:
# Adding city and smart_location to cols_del
cols_del.update(['city'],['smart_location'])

Exploring the host listing count columns: calculated_host_listings_count, host_total_listings_count, host_listings_count

In [19]:
data[['host_id','calculated_host_listings_count','host_total_listings_count','host_listings_count']].head()

,host_id,calculated_host_listings_count,host_total_listings_count,host_listings_count
0,2758,3,3.0,3.0
1,42032,1,1.0,1.0
2,2787,8,8.0,8.0
3,2845,2,5.0,5.0
4,82685,1,1.0,1.0


In [20]:
data[data['host_total_listings_count']!=data['host_listings_count']]

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
6840,4446862,https://www.airbnb.com/rooms/4446862,20181206022948,2018-12-06,Charming Room in Prospect Heights!,Quiet room in three bedroom apt in Prospect He...,Enjoy your stay in a clean and quiet bedroom i...,Quiet room in three bedroom apt in Prospect He...,none,This charming Brooklyn neighborhood offers man...,NaN,Close to the 2/3/4/5/B/Q trains you are not bu...,Full access to kitchen stocked with coffee and...,Will coordinate time to meet and greet guest!,"Please be respectful of the space, the cats, a...",NaN,NaN,https://a0.muscache.com/im/pictures/57579076/a...,NaN,23077718,https://www.airbnb.com/users/show/23077718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,"Brooklyn, NY, United States",Crown Heights,Crown Heights,Brooklyn,Brooklyn,NY,11238,New York,"Brooklyn, NY",US,United States,40.675120,-73.961458,t,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets live...",NaN,$50.00,NaN,NaN,NaN,NaN,1,$0.00,1,1125,43 months ago,t,0,0,0,0,2018-12-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,flexible,f,f,1,NaN
7458,4763327,https://www.airbnb.com/rooms/4763327,20181206022948,2018-12-06,"Luxurious, best location, spa inc'l","Brand new luxurious boutique building, swimmin...","APARTMENT: - 1 bedroom apartment - Modern, bra...","Brand new luxurious boutique building, swimmin...",none,"Quoting from Time Out Magazine: ""Williamsburg ...",NaN,"This is incredibly convenient location. First,...",Whole apartment and all the building amenities...,I am available 24/7 to support and will provid...,Have fun! :),NaN,NaN,https://a0.muscache.com/im/pictures/59917673/e...,NaN,24576978,https://www.airbnb.com/users/show/24576978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,"Brooklyn, NY, United States",Williamsburg,Greenpoint,Brooklyn,Brooklyn,NY,11211,New York,"Brooklyn, NY",US,United States,40.720345,-73.953554,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$195.00,"$1,200.00","$4,000.00",$295.00,$70.00,2,$25.00,1,1125,38 months ago,t,0,0,0,0,2018-12-06,1,2015-10-20,2015-10-20,80.0,8.0,10.0,10.0,8.0,10.0,8.0,f,NaN,NaN,f,f,moderate,f,f,1,0.03
9304,6360224,https://www.airbnb.com/rooms/6360224,20181206022948,2018-12-06,"Sunny, Private room in Bushwick","Spacious room, lots of light. New, clean build...",NaN,"Spacious room, lots of light. New, clean build...",none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/87714659/5...,NaN,33134899,https://www.airbnb.com/use

In [21]:
#Checking what calculated_host_listings_count represents.
([sum(data['host_id'] == data['host_id'][i]) for i in range(len(data))]==data['calculated_host_listings_count']).value_counts()


True    49056
Name: calculated_host_listings_count, dtype: int64

Based on the analysis above we can conclude:<br>
>host_total_listings_count and host_listings_count columns are same except for 7 rows where the value in NaN. For the NaN values, we see that all other host columns are also null. This indicates that maybe its a new host with no initial data available. Since there are only 7 such listings, we can drop those listings. Also, since the columns are redundat we will only keep one of the two. <br>
For calculated_host_listings_count column, we find out that it contains the number of listings for that particular host in the given dataset. It is unecessary at this point since we have the columns with total listings a host has, which would be more informative.

In [22]:
# Adding the 2 columns to be dropped to cols_del:
cols_del.update(['calculated_host_listings_count','host_listings_count'])

In [23]:
data['street'].head()

0     New York, NY, United States
1     New York, NY, United States
2    Brooklyn , NY, United States
3     New York, NY, United States
4     New York, NY, United States
Name: street, dtype: object

The street column has the format city,state,country and we have those columns available independently so we can delete the street column from the dataset.

In [24]:
cols_del.add('street')

In [25]:
data['state'].value_counts()

NY         49028
Ny             7
ny             4
MP             1
Paracas        1
CA             1
NJ             1
Name: state, dtype: int64

On evaluation of the state column we see that NY is stored in different formats and there are listings from other states as well. We can delete the listings from CA, Paracas, NJ and MP. Once that is accomplished, we can delete the state coulmn since the value would be consistent for all remaining listings - NY.

In [26]:
cols_del.add('state') #adding state column to cols_del

In [27]:
data['market'].value_counts()

New York                       48897
Other (Domestic)                  18
Adirondacks                        2
Los Angeles                        1
South Bay, CA                      1
Lagos, NG                          1
Kyoto                              1
Catskills and Hudson Valley        1
Cuba                               1
D.C.                               1
Atlanta                            1
Paris                              1
Agra                               1
London                             1
Jamaica South Coast                1
San Francisco                      1
Boston                             1
New Orleans                        1
Name: market, dtype: int64

On evaluating the market column, we find that most of the listings are in New York market, but there are other market values listed as well. We can delete the rows with any market other than New York, and then we can drop the market column since it will have the same value consistently for all listings.

In [28]:
cols_del.add('market') #adding market column to set of columns to be dropped

In [29]:
print(data['country'].value_counts())
print(data['country_code'].value_counts())

United States    49055
Peru                 1
Name: country, dtype: int64
US    49055
PE        1
Name: country_code, dtype: int64


One of the listings in apparently in Peru and will need to be deleted.
Otherwise, Columns country and country_code have consistent values for all listings. So they can be dropped.

In [30]:
#Adding the above mentioned two columns to cols_del
cols_del.update(['country','country_code'])

In [31]:
data=data[~data['description'].isnull()]

In [32]:
data=data[~data['host_total_listings_count'].isnull()]

In [33]:
drop_states=['CA','NJ','MP'] 
data=data[~data['state'].isin(drop_states)] #dropping rows having different states listed.

In [34]:
data=data[data['market']=='New York'] #drop rows with different marrket values

In [35]:
data=data.reset_index()

Dropping the list of columns from the dataframe

In [36]:
cols_del_list=list(cols_del)

In [37]:
data.drop(cols_del_list, axis=1, inplace=True)

In [38]:
data.drop('index', axis=1, inplace=True)

#### The following is the cleaned up dataframe, where the unnecessary columns have been deleted and few row with missing data or incunsistent data have also been dropped.

In [39]:
data.head()

,name,description,neighborhood_overview,transit,access,interaction,house_rules,host_id,host_since,host_location,host_about,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month
0,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,NaN,Subway 2.3.B.C. at 110th street around the cor...,Guests will have their PRIVATE BATHROOM (NOTE:...,We will have a list of Harlem restaurants and ...,no-smoking/please take off your shoes: cleanin...,2758,2008-09-06,"New York, New York, United States",loves to host and welcome travelers from arou...,within a few hours,70%,t,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,Harlem,Manhattan,10026,40.799205,-73.953676,t,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$59.00,$0.00,$60.00,2,$39.00,2,21,today,3,12,23,298,175,2008-10-13,2018-11-18,93.0,9.0,9.0,10.0,9.0,9.0,9.0,f,strict_14_with_grace_period,f,f,1.42
1,Light-filled classic Central Park,"An adorable, classic, clean, light-filled one-...","Diverse. Great coffee shops and restaurants, n...","Yes! B, C, and 1 trains are within 5 minute w...",Guests have access to the whole apartment: wi...,I interact with guests upon arrival & departur...,"Sorry, no pets and no smoking, please. No pa...",42032,2009-09-29,"New York, New York, United States","I'm an arts consultant, personal trainer and a...",within a day,80%,f,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,Upper West Side,Manhattan,10025,40.797642,-73.961775,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",$140.00,NaN,$40.00,2,$28.00,5,365,yesterday,8,8,13,248,76,2010-03-16,2018-10-16,94.0,10.0,9.0,10.0,10.0,10.0,10.0,f,moderate,t,t,0.72
2,Clean & quiet apt home by the park,Renovated apt home in elevator building. Spaci...,Close to Prospect Park and Historic Ditmas Park,Very close to F and G trains and Express bus i...,NaN,NaN,-The security and comfort of all our guests is...,2787,2008-09-07,"New York, New York, United States",Educated professional living in Brooklyn. I l...,within an hour,100%,t,8.0,"['email', 'phone', 'reviews', 'kba']",t,t,Kensington,Brooklyn,11218,40.647486,-73.972370,f,Apartment,Private room,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Wheelchair acces...",$149.00,$100.00,$25.00,1,$25.00,1,730,today,30,60,90,365,9,2015-12-04,2018-10-19,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,f,f,0.25
3,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...",Centrally located in the heart of Manhattan ju...,Apartment is located on 37th Street between 5t...,"Guests have full access to the kitchen, bathro...",I am a Sound Therapy Practitioner and Kundalin...,"Make yourself at home, respect the space and t...",2845,2008-09-09,"New York, New York, United States",A New Yorker since 2000! My passion is creatin...,within a few hours,82%,f,5.0,"['email', 'phone', 'reviews', 'kba', 'work_ema...",t,t,Midtown,Manhattan,10018,40.753621,-73.983774,f,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",$225.00,$350.00,$100.00,2,$0.00,1,1125,3 days ago,15,45,75,350,41,2009-11-21,2018-12-02,95.0,9.0,9.0,10.0,10.0,10.

In [40]:
data.shape

(48353, 59)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48353 entries, 0 to 48352
Data columns (total 59 columns):
name                                48339 non-null object
description                         48353 non-null object
neighborhood_overview               29797 non-null object
transit                             30833 non-null object
access                              28456 non-null object
interaction                         27743 non-null object
house_rules                         29358 non-null object
host_id                             48353 non-null int64
host_since                          48353 non-null object
host_location                       48208 non-null object
host_about                          30379 non-null object
host_response_time                  26795 non-null object
host_response_rate                  26795 non-null object
host_is_superhost                   48353 non-null object
host_total_listings_count           48353 non-null float64
host_verifications   

As discussed before, we will simplify columns neighbourhood overview, transit, access, interastion and house rules. If the information was provided it would be classified as provided and if it contains NaN then it will be classsified as not provided

In [42]:
data['neighborhood_overview']=data['neighborhood_overview'].apply(lambda x: 'Provided' if type(x)==str else 'Not Provided')
data['transit']=data['transit'].apply(lambda x: 'Provided' if type(x)==str else 'Not Provided')
data['access']=data['access'].apply(lambda x: 'Provided' if type(x)==str else 'Not Provided')
data['interaction']=data['interaction'].apply(lambda x: 'Provided' if type(x)==str else 'Not Provided')
data['house_rules']=data['house_rules'].apply(lambda x: 'Provided' if type(x)==str else 'Not Provided')


For the remaining text columns including name, description and host_about. We will combine those columns as one column which will contain the combined text from all three columns and we will analyze it as a whole. 

In [43]:
data['name_describe_host'] = data['name'].str.cat(data['description'],sep=" ").str.cat(data['host_about'],sep=" ")

Now, we can drop the individual columns from the dataframe.

In [44]:
data.drop(['name','description','host_about'], axis=1, inplace=True)

For the host_location column- we will classify whether the host is located in NY, In US but not in NY or outside US using the following function. The null values will be considered as location not provided.

In [45]:
def location_classifier(loc_string):
    try:
        if ('New York' in loc_string or 'NY' in loc_string):
            return ('In NY')
        elif ('United States' in loc_string or 'US' in loc_string):
            return ('In US but not in NY')
        else:
            return ('Not in US')
    except:
        return ('Not Provided')

In [46]:
data['host_location']=data['host_location'].apply(lambda x: location_classifier(x))

The host_response time has several NaN values, we assume that the host did not respond in those cases.

In [47]:
data.host_response_time.fillna('did not respond', inplace=True)

The host response rate column includes several NaN values, and we can assume that in those cases, the response rate was 0%. We will fill in the null values with 0 and convert the % to float values using the following function:

In [48]:
def percent_to_float(percent):
    return float(percent.strip('%'))/100

In [49]:
data.host_response_rate.fillna('0%', inplace=True)
data['host_response_rate']=data['host_response_rate'].apply(lambda x: percent_to_float(x))

In [50]:
data.head()

,neighborhood_overview,transit,access,interaction,house_rules,host_id,host_since,host_location,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month,name_describe_host
0,Not Provided,Provided,Provided,Provided,Provided,2758,2008-09-06,In NY,within a few hours,0.70,t,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,Harlem,Manhattan,10026,40.799205,-73.953676,t,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$59.00,$0.00,$60.00,2,$39.00,2,21,today,3,12,23,298,175,2008-10-13,2018-11-18,93.0,9.0,9.0,10.0,9.0,9.0,9.0,f,strict_14_with_grace_period,f,f,1.42,Stay at Chez Chic budget room #1 Step into our...
1,Provided,Provided,Provided,Provided,Provided,42032,2009-09-29,In NY,within a day,0.80,f,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,Upper West Side,Manhattan,10025,40.797642,-73.961775,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",$140.00,NaN,$40.00,2,$28.00,5,365,yesterday,8,8,13,248,76,2010-03-16,2018-10-16,94.0,10.0,9.0,10.0,10.0,10.0,10.0,f,moderate,t,t,0.72,Light-filled classic Central Park An adorable...
2,Provided,Provided,Not Provided,Not Provided,Provided,2787,2008-09-07,In NY,within an hour,1.00,t,8.0,"['email', 'phone', 'reviews', 'kba']",t,t,Kensington,Brooklyn,11218,40.647486,-73.972370,f,Apartment,Private room,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Wheelchair acces...",$149.00,$100.00,$25.00,1,$25.00,1,730,today,30,60,90,365,9,2015-12-04,2018-10-19,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,f,f,0.25,Clean & quiet apt home by the park Renovated a...
3,Provided,Provided,Provided,Provided,Provided,2845,2008-09-09,In NY,within a few hours,0.82,f,5.0,"['email', 'phone', 'reviews', 'kba', 'work_ema...",t,t,Midtown,Manhattan,10018,40.753621,-73.983774,f,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",$225.00,$350.00,$100.00,2,$0.00,1,1125,3 days ago,15,45,75,350,41,2009-11-21,2018-12-02,95.0,9.0,9.0,10.0,10.0,10.0,9.0,f,strict_14_with_grace_period,t,t,0.37,Skylit Midtown Castle Find your romantic getaw...
4,Provided,Provided,Provided,Provided,Provided,82685,2010-02-18,In NY,did not respond,0.00,f,1.0,"['email', 'phone', 'google', 'reviews', 'jumio...",t,t,Harlem,Manhattan,10031,40.828028,-73.947308,t,Apartment,Private room,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,E...",$89.00,$100.00,$30.00,1,$55.00,1,60,29 months ago,30,60,90,365,1,2018-10-09,2018-10-09,100.0,10.0,10.0,10.0,10.0,10.0,8.0,f,strict_14_with_grace_period,f,f,0.51,"Upper Manhattan, New York A great space in a b..."


For missing values in the columns first and last review, we notice that those are the listings for which number of reviews=0. So, the missing values can be filled with NaT.<br>

As for all the other columns with missing values, including bathroom, bedrooms, security deposit, cleaning fee, and all different rating columns which have null values can be filled in with 0.

In [51]:
data['first_review'].fillna(pd.NaT, inplace=True)
data['last_review'].fillna(pd.NaT, inplace=True)

In [52]:
data['bathrooms'].fillna(0, inplace=True)
data['bedrooms'].fillna(0, inplace=True)
data['beds'].fillna(0, inplace=True)
data['security_deposit'].fillna(0, inplace=True)
data['cleaning_fee'].fillna(0, inplace=True)
data['review_scores_accuracy'].fillna(0, inplace=True)
data['review_scores_checkin'].fillna(0, inplace=True)
data['review_scores_cleanliness'].fillna(0, inplace=True)
data['review_scores_communication'].fillna(0, inplace=True)
data['review_scores_location'].fillna(0, inplace=True)
data['review_scores_rating'].fillna(0, inplace=True)
data['review_scores_value'].fillna(0, inplace=True)
data['reviews_per_month'].fillna(0,inplace=True)

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48353 entries, 0 to 48352
Data columns (total 57 columns):
neighborhood_overview               48353 non-null object
transit                             48353 non-null object
access                              48353 non-null object
interaction                         48353 non-null object
house_rules                         48353 non-null object
host_id                             48353 non-null int64
host_since                          48353 non-null object
host_location                       48353 non-null object
host_response_time                  48353 non-null object
host_response_rate                  48353 non-null float64
host_is_superhost                   48353 non-null object
host_total_listings_count           48353 non-null float64
host_verifications                  48353 non-null object
host_has_profile_pic                48353 non-null object
host_identity_verified              48353 non-null object
neighbourhood_cleans

# Data dictionary for the remaining columns to help determine which ones would be useful features:
> id - unique identifier number for the listing <br>
listing_url - url where the listing is found (contains the id) <br>
scrape_id - ??? <br>
last_scraped - ??? <br>
name - name of the property	<br>
summary	- summary of the property provided by the host<br>
space - description of space provided under the title 'The Space'	<br>
description	- summary + space text put together<br>
experiences_offered	- details/description of expereinces offered at the property<br>
neighborhood_overview - brief description of the neighborhood<br>
transit	- transit options avialable close to the property<br>
access - description proovided under the title 'Guest Access'<br>
interaction	- description of host's interaction with the customer<br>
house_rules	- house rules set by the host (listed under policy --> house rules)<br>
picture_url	- url for the picture of property<br>
host_id	- unique ID number for each host<br>
host_url - url where host's profile can be viewed (contains host id)<br>
host_name - name of the host<br>
host_since - date when started hosting on Airbnb<br>
host_location - where the host is located<br>
host_about - brief description provided about themselves by the host<br>
host_response_time - usual response time of the host to customers<br>
host_response_rate - how often the host responds (in %)<br>
host_is_superhost - whether host is a superhost (true/false)<br>
host_thumbnail_url - url to host's thumbnail<br>
host_picture_url - url to host's picture<br>
host_neighbourhood - host's neighborhood<br>
host_listings_count	- number of listings the host has<br>
host_total_listings_count - total listings of the host's properties<br>
host_verifications - forms of verifications of host available<br>
host_has_profile_pic - whether the host has a profile picture<br>
host_identity_verified - whether the host's identity has been verified<br>
street - ???	<br>
neighbourhood - ???	<br>
neighbourhood_cleansed - ???<br>
neighbourhood_group_cleansed - ???<br>
city - city of the property<br>
state - state of the property<br>
zipcode	- zipcode of the property<br>
market - ???<br>
smart_location - ???<br>
country_code - country code (US)	<br>
country	- country name<br>
latitude - lattitude of the property	<br>
longitude - longitude of the property<br>
is_location_exact - whether the location provided is exact (true/false)<br>
property_type - type of property<br>
room_type - entire apartment / private room / shared room<br>
accommodates - how many people can be accomodated<br>
bathrooms - number of bathrooms<br>
bedrooms - number of bedrooms<br>
beds - number of beds<br>
bed_type - type of bed available<br>
amenities - available amenities	<br>
price - price of airbnb per night<br>
security_deposit - amount of security deposit required<br>
cleaning_fee - cleaning fee<br>
guests_included	- how many guests are included in the cost<br>
extra_people - cost of adding an extra person<br>
minimum_nights - minimum number of nights the customer has to book<br>
maximum_nights - maximum number of nights the customer can book<br>
calendar_updated - when was the calender on the listing last updated<br>	
has_availability - whether there is availability (true/false)<br>
availability_30	- number of nights available in the next 30 days (from date when it was scraped 12/6/18)<br>
availability_60	- number of nights available in the next 60 days (from date when it was scraped 12/6/18)<br>
availability_90	- number of nights available in the next 90 days (from date when it was scraped 12/6/18)<br>
availability_365 - number of nights available in the next 365 days (from date when it was scraped 12/6/18)<br>
calendar_last_scraped - date when the calender was last scraped<br>
number_of_reviews - number of reviews for the listing<br>
first_review - date of first review	<br>
last_review	- date of last review<br>
review_scores_rating - overall rating (out of 100)	<br>
review_scores_accuracy	- rating for accuracy (out of 10)<br>
review_scores_cleanliness - rating for cleanliness (out of 10)<br>
review_scores_checkin - rating for checkin (out of 10)<br>
review_scores_communication	- rating for communication (out of 10)<br>
review_scores_location - rating for location (out of 10)<br>
review_scores_value	- rating for value (out of 10)<br>
requires_license - whether license is required (true/false)<br>
instant_bookable - whether it is instantly bookable (true/false)<br>
is_business_travel_ready - whether it is business travel ready (true/false)	<br>
cancellation_policy	- description of the cancellation policy<br>
require_guest_profile_picture - whether guest profice picture is required (true/false)	<br>
require_guest_phone_verification - whether guest phone verification is required (true/false)<br>
calculated_host_listings_count - number of listings by the host<br>
reviews_per_month - average number of reviews per month